<a href="https://colab.research.google.com/github/tanongsakintean/google_colab/blob/main/deployModelAsl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 71 bytes


In [3]:
!kaggle datasets download -d grassknoted/asl-alphabet

100% 1.02G/1.03G [00:09<00:00, 66.9MB/s]
100% 1.03G/1.03G [00:09<00:00, 119MB/s] 


In [4]:
import zipfile
from keras.preprocessing.image import ImageDataGenerator
zip_ref = zipfile.ZipFile('asl-alphabet.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
from google.colab import drive #case mounted gdrive
drive.mount('/content/gdrive', force_remount=False)


Mounted at /content/gdrive


In [6]:
import pickle
pickled_model = pickle.load(open('/content/gdrive/MyDrive/projectAiModel/asl_cnn_new.pkl', 'rb'))
#pickled_model.predict(X_test)
#summaize model
pickled_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 64, 64, 32)        0         
 ng2D)                                                           
                                                                 
 batch_normalization_15 (Ba  (None, 64, 64, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_31 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 32, 32, 64)        0         
 ng2D)                                                           
                                                      

In [7]:
train_gen=ImageDataGenerator(rescale=1./255,validation_split=0.2)
train_data=train_gen.flow_from_directory("/content/asl_alphabet_train/asl_alphabet_train",target_size=(200,200),batch_size=32,class_mode='categorical',shuffle=True,subset='training')

Found 69600 images belonging to 29 classes.


In [8]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
import matplotlib.pyplot as plt
import numpy as np

In [9]:
# load and prepare the image
def load_image(filename):
 # load the image
 img = load_img(filename, target_size=(128, 128))
 # convert to array
 img = img_to_array(img)
 # reshape into a single sample with 3 channels
 img = img.reshape(1, 128, 128, 3)
 # center pixel data
 img = img.astype('float32')
 img /= 255
 #img = img - [123.68, 116.779, 103.939]
 return img

def show_img(path):
  image = load_img(path)
  plt.imshow(image)

# load an image and predict the class
def run_example(path, model):
 # load the image
 img = load_image(path)
 # predict the class
 result = model.predict(img)
 print(result)
#  result_class = (result> 0.5).astype("int32")
 show_img(path)
 class_indices_array = np.array(list(train_data.class_indices.keys()))
 print("predicted label:",class_indices_array[np.argmax(result[0])])


In [ ]:
run_example('/content/Q.png',pickled_model)

In [10]:
pip install flask-ngrok

In [11]:
cd /content/gdrive/MyDrive/demo_deploy/dist

/content/gdrive/MyDrive/demo_deploy/dist


In [12]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pickle
import numpy as np

In [13]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=e2372ed0499a587e05ef8c1dc53eb639161e19be3be59249570e9776b87eee51
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [14]:
!ngrok authtoken 2WeTsMZvjX0yVGJd3ctNf1juDAJ_gfvSd39j6VEkqBWLsQVJ

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [22]:
from keras.models import load_model

app = Flask(__name__)

run_with_ngrok(app)

model = pickle.load(open('asl_cnn_new.pkl','rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=["POST"])

def predict():
    if request.method == 'POST':
        petal_length = request.form['petal_length']
        sepal_length = request.form['sepal_length']
        petal_width = request.form['petal_width']
        sepal_width = request.form['sepal_width']

        sample_data = [sepal_length, sepal_width, petal_length, petal_width]
        clean_data = [float(i) for i in sample_data]

        final_input = np.array(clean_data).reshape(1,-1)
        print(final_input)

        prediction = model.predict(final_input)
        class_unseen=np.argmax(prediction,axis=-1) #majority vote from the prediction
        print("predictedclass ",class_unseen)

    return render_template('index.html',result_prediction=class_unseen)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://6372-34-73-241-201.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [08/Nov/2023 06:12:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Nov/2023 06:12:03] "GET /favicon.ico HTTP/1.1" 404 -
